In [1]:
import os
import sys
#path environment
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")
import traci

import xml.etree.ElementTree as ET

In [2]:
sumo_cmd = ['sumo-gui', 
            '--duration-log.statistics', 
            '--tripinfo-output', 'my_output_file.xml', 
            '-c', 'maps/grid/grid.sumo.cfg']

In [3]:
def analyze_time_loss(file_path):
    # Parse the XML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Initialize variables for analysis
    total_time_loss = 0.0
    total_waiting_time = 0.0
    total_duration = 0.0
    total_reroutes = 0
    vehicle_count = 0

    # Iterate over each tripinfo element in the XML
    for trip in root.findall('tripinfo'):
        time_loss = float(trip.get('timeLoss'))
        waiting_time = float(trip.get('waitingTime'))
        duration = float(trip.get('duration'))
        reroutes = int(trip.get('rerouteNo'))
        
        total_time_loss += time_loss
        total_waiting_time += waiting_time
        total_duration += duration
        total_reroutes += reroutes
        vehicle_count += 1

    # Calculate averages
    average_time_loss = total_time_loss / vehicle_count if vehicle_count > 0 else 0
    average_waiting_time = total_waiting_time / vehicle_count if vehicle_count > 0 else 0
    average_duration = total_duration / vehicle_count if vehicle_count > 0 else 0
    average_reroutes = total_reroutes / vehicle_count if vehicle_count > 0 else 0

    # Store results in a dictionary
    results = {
        'Total Vehicles': vehicle_count,
        'Total Time Loss': total_time_loss,
        'Average Time Loss': average_time_loss,
        'Total Waiting Time': total_waiting_time,
        'Average Waiting Time': average_waiting_time,
        'Average Duration': average_duration,
        'Total Reroutes': total_reroutes,
        'Average Reroutes': average_reroutes
    }
    return results


In [4]:
# Start SUMO with TraCI support
traci.start(sumo_cmd)
step = 0
while step < 3600:
    traci.simulationStep()
    
    
    step += 1

traci.close()


 Retrying in 1 seconds


In [5]:
results = analyze_time_loss('my_output_file.xml')


In [6]:
results

{'Total Vehicles': 4353,
 'Total Time Loss': 183857.10000000068,
 'Average Time Loss': 42.23687112336335,
 'Total Waiting Time': 83129.0,
 'Average Waiting Time': 19.0969446358833,
 'Average Duration': 114.82609694463588,
 'Total Reroutes': 0,
 'Average Reroutes': 0.0}